# 스크롤 제어

스크롤을 제어하기 위해선 javascript가 필요한데 이를 selenium에서 제공하는 `execute_script` 메서드를 사용해서 javascript를 실행해 스크롤을 제어할 수 있다.

- window.scrollTo(0, 100): 절대 위치(100px)로 스크롤
- window.scrollBy(0, 100): 상대 위치(100px)로 스크롤
- element.scrollIntoView(): 해당 요소의 위치까지 스크롤

execute_script은 그냥 안에 javascript code를 작성해도 된다.
- execute_script에 실행되는 script를 보면 익명 함수처럼 실행되는 느낌 같음
- 따라서, 인자를 받는데 javascript 기본동작에 의해서 arguments 리스트를 통해 접근이 가능하다.
- python의 f-string을 통해서 특정 값을 받고 다시 스크립트에 넣는 것도 가능해보임.

```python
js_script = """
function sum(a, b) {
  return a + b;
}

return sum(arguments[0], arguments[1])
"""

get_sum_result = driver.execute_script(js_script, 1, 2)
```


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



URL = "https://startcoding.pythonanywhere.com/dynamic"
driver = webdriver.Chrome()

driver.get(URL)

# * 생각해보니까 동적 페이지는 모든 콘텐츠가 loading이 완료된 후 하는게 안전한 것 같다.
# * document.readyState
# * - "loading": 파싱 중
# * - "interactive": DOM 만들어짐 (= DOMContentLoaded 시점이랑 거의 비슷)
# * - "complete": DOM도 끝, 이미지·CSS 등 리소스도 거의 다 끝
# * 따라서 readyState == "complete" 일때 크롤링을 진행하면 안전할 것 같다.
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")

# ? 근데 다시 생각해보니 javascript는 추가 요청을 통해 resource를 불러오므로 차라리 원하는 content가 보일 때까지 대기하는 것이 더 안정적인 것 같음
# WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.CSS_SELECTOR, ".list-item"))
# )

## 절대 위치로 스크롤

In [14]:
move_absolute_position_scroll_script ="""
window.scrollTo(0, 2000)
"""
driver.execute_script(move_absolute_position_scroll_script)

## 상대 위치로 스크롤

In [38]:
move_relative_position_scroll_script ="""
window.scrollBy(0, 500)
"""
driver.execute_script(move_relative_position_scroll_script)

## 끝까지 스크롤

In [40]:
total_height = driver.execute_script("return document.body.scrollHeight")
scroll_to_bottom_script = f"""
window.scrollTo(0, {total_height})
"""

driver.execute_script(scroll_to_bottom_script)

## 특정 요소로 스크롤

In [52]:
target_element = driver.find_element(By.CSS_SELECTOR, "#product-container > div:nth-of-type(6)")
driver.execute_script("""
const targetElement = arguments[0];
targetElement.scrollIntoView(targetElement);
""", target_element )

In [50]:
driver.quit()

## python과 javascript 연계 -> 잘 됨!

In [41]:
test_script = """
const sum = (a, b) =>  a + b;

return sum(arguments[0], arguments[1])
"""

result = driver.execute_script(test_script, 1, 2)
print(f"result: {result}")

result: 3
